In [240]:
import os
import pandas as pd
import wget
import sys
import subprocess
from subprocess import call

import json

import numpy as np

from metamind.api import ClassificationData, ClassificationModel, set_api_key

sys.path.append(os.environ["HOME"])
from ds_config import *

print os.environ["HOME"]
MAIN_PATH = os.environ["HOME"]+"/GLaDOS_Workspace/"
#os.mkdir(MAIN_PATH)
#os.mkdir(MAIN_PATH+"Videos/")
#os.mkdir(MAIN_PATH+"Images/")

/Users/williamspearman


In [190]:
arr = [ 1, 3, 1245, 346 ]

[ i for x, i in zip(arr, range(0,len(arr))) ]

[0, 1, 2, 3]

# FFMPEG Scrips

In [309]:
def getVideoInfo(file_name):
    bashCommand = "ffprobe -show_streams " + file_name
    process = subprocess.Popen(bashCommand.split(" "), stdout=subprocess.PIPE)
    output = process.communicate()[0].split("\n")
    ret_dict = { }
    for x in output:
        if "=" in x:
            key_vals = x.split("=")
            ret_dict[key_vals[0]] = key_vals[1]
    #print ret_dict
    return ret_dict

def getRandInt(int1, int2):
    if np.random.random() > 0.5:
        return np.random.randint(int1[0],int1[1])
    else:
        return np.random.randint(int2[0],int2[1])


def getGetSignalBkgTriplets(file_name, snap_time, pm_secs=1.):
    ret_val = { }
    info_dict = getVideoInfo(file_name)
    required_vals = { "duration", "nb_frames", "avg_frame_rate", "sample_aspect_ratio" }
    if len(required_vals.intersection(set(info_dict.keys())))!=len(required_vals):
        return None
    frame_rate = 0.
    try:
        frame_rate = float(info_dict["avg_frame_rate"].split("/")[0])/float(info_dict["avg_frame_rate"].split("/")[1])
    except:
        return None
    num_frames = float(info_dict["nb_frames"])
    duration = float(info_dict["duration"])

    if int(num_frames/frame_rate-duration)!= 0:
        return None
    
    pm_frames = pm_secs * frame_rate
    snap_frame = snap_time * frame_rate
    
    snap_frame_low = snap_frame-pm_frames
    snap_frame_high = snap_frame+pm_frames
    if snap_frame_low<0:
        return None
    if snap_frame_high>num_frames:
        return None
    
    int1 = [ pm_frames, max(0,snap_frame_low-pm_frames) ]
    int2 = [ min(pm_frames+snap_frame_high, num_frames), num_frames-pm_frames ]
    bkg_frame = getRandInt(int1, int2)
    
    ret_val["SIGNAL"] = [ snap_frame_low/frame_rate, snap_frame/frame_rate, snap_frame_high/frame_rate ]
    ret_val["BACKGROUND"] = [ (bkg_frame-pm_frames)/frame_rate, bkg_frame/frame_rate, (bkg_frame+pm_frames)/frame_rate ]
    
    return ret_val
    
def downloadUrl(url):
    base_name = ""
    try:
        base_name = url.split("/")[-1]
    except:
        return ""
    
    output_name = MAIN_PATH + "Videos/" + base_name
    if os.path.isfile(output_name)==False:
        wget.download(url, MAIN_PATH+"Videos/")
    
    if os.path.isfile(output_name)==True:
        return output_name
    else:
        return ""



def makeImageMontage(url, snap_time, time_offset=0.1):
    ret_val = [ ]
    video_path = downloadUrl(url)
    #print "BLah %f" % 0.2
    base_name = ""
    try:
        base_name = video_path.split("/")[-1].split(".")[0]
    except:
        print "Failed splitting: %s" % video_path
        return None
    input_times = getGetSignalBkgTriplets(video_path, snap_time, time_offset)
    if input_times==None:
        return ret_val
    #resolution = resolution.replace(":", "x")
    #name1 = "blah"
    #name2 = "blah"
    ffmpeg_cmds = { }
    file_names = { }
    for mykey in input_times.keys():
        output_name = MAIN_PATH + "Images/" + base_name
        ffmpeg_cmds[mykey] = [ ]
        file_names[mykey] = [ ]
        for x, i in zip(input_times[mykey], range(0,len(input_times[mykey]))):
            file_name = "%s_%i_%s.jpg" % (output_name, i, mykey)
            file_names[mykey].append(file_name)
            ffmpeg_cmds[mykey].append("ffmpeg -ss %f -i %s -vframes 1 %s 2> /dev/null" % (x, video_path, file_name))
        #montage_cmd = "montage -tile 1x3 -geometry 512x256+0+0 "
        montage_cmd = "montage -tile 1x3 -geometry 512x+0+0 "
        for thecmd, fname in zip(ffmpeg_cmds[mykey], file_names[mykey]):
            call(thecmd, shell=True)
            montage_cmd = montage_cmd+fname+" "
        montage_name = "%s_%i_%s_MONTAGE.jpg" % (output_name, int(1000*input_times[mykey][1]), mykey)
        montage_cmd = montage_cmd + " " + montage_name
        call(montage_cmd,shell=True)
        for fname in file_names[mykey]:
            os.remove(fname)
        ret_val.append((montage_name, mykey))
        
        
        
            
    return ret_val


In [302]:
print makeImageMontage("http://vf.hudl.com/9n6/81k/bab387d7-c875-4df6-89f7-3a0c910deb6a/lelyp36kuaykgm1j-3000_1100.mp4", 8.809, 0.5)

[('/Users/williamspearman/GLaDOS_Workspace/Images/lelyp36kuaykgm1j-3000_1100_8809_SIGNAL_MONTAGE.jpg', 'SIGNAL'), ('/Users/williamspearman/GLaDOS_Workspace/Images/lelyp36kuaykgm1j-3000_1100_16900_BACKGROUND_MONTAGE.jpg', 'BACKGROUND')]


In [310]:
snapRDD = sc.textFile("s3n://hudl-hadoop/GLaDOS/snap_detect_data/").map(json.loads)

In [325]:
snapRDD_size = snapRDD.count()
trainNum = 100.
trainNum = 20.

snapRDD_train = snapRDD.sample(False, trainNum/snapRDD_size).cache()
snapRDD_test = snapRDD.sample(False, trainNum/snapRDD_size).cache()
print snapRDD_train.count()
print snapRDD_test.count()

23
15


In [315]:
training_inputRDD = snapRDD_small\
    .flatMap(lambda x: makeImageMontage(x["video"]["urls"][-1],x["video"]["snap time"]))
training_inputs = training_inputRDD.collect()

In [ ]:
test_inputRDD = snapRDD_small\
    .flatMap(lambda x: makeImageMontage(x["video"]["urls"][-1],x["video"]["snap time"]))
test_inputs = training_inputRDD.collect()

In [320]:
set_api_key(metamind_api_key)

Hello, William Spearman


/Users/williamspearman/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [321]:
training_data = ClassificationData(private=True, data_type="image", name="snap_detect")

You are creating dataset 'snap_detect', with id: 152211


/Users/williamspearman/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [322]:
training_data.add_samples(training_inputs, input_type="files")

Starting data upload...
Finished uploading 150 of 182 samples...

/Users/williamspearman/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/Users/williamspearman/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning



Finished uploading 182 of 182 samples...


In [323]:
classifier = ClassificationModel(private=True, name="snap_detect")
classifier.fit(training_data)

/Users/williamspearman/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/Users/williamspearman/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/Users/williamspearman/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents

Your MetaMind image model is now training on the server!
Estimated time left in training: 0m 24s

/Users/williamspearman/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/Users/williamspearman/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning



Estimated time left in training: 0m 21s

/Users/williamspearman/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/Users/williamspearman/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


...

/Users/williamspearman/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/Users/williamspearman/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/Users/williamspearman/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents


Estimated time left in training: 0m 4s

/Users/williamspearman/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/Users/williamspearman/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning



Estimated time left in training: 0m 2s
Your model is done training!
Model name: snap_detect
Model id: 18406


/Users/williamspearman/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [77]:
classifier.stats()

/home/ubuntu/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


{'time_left': 3.6428284645080566,
 'training_stats': {u'cost': [1.2934340953826904,
   1.4668760538101195,
   1.392206573486328,
   1.3992962837219238,
   1.2664953947067261,
   1.3129367589950562,
   1.2972320079803468,
   1.335271453857422,
   1.3376054763793945,
   1.3716208457946777,
   1.246858286857605,
   1.2130812168121339,
   1.334204339981079,
   1.3556113243103027,
   1.2497127056121826,
   1.232087540626526,
   1.1976725339889527,
   1.243276834487915,
   1.3334039211273194,
   1.1425410747528075,
   1.271357822418213,
   1.2586041688919067,
   1.2652908086776733,
   1.2556301593780517,
   1.2081805229187013,
   1.270026397705078,
   1.154785680770874,
   1.1764276266098022,
   1.1093882620334625,
   1.1200138330459595,
   1.2423322796821594,
   1.2723020911216736,
   1.2030891478061676,
   1.1919867098331451,
   1.1837872564792633,
   1.156642034649849,
   1.1856812834739685,
   1.1259594857692719,
   1.2325102984905243,
   1.0912235230207443,
   1.2793141901493073,
   1.2

In [88]:
preds = classifier.predict(fg_test_set, input_type="files")

/home/ubuntu/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/home/ubuntu/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
Traceback (most recent call last):
  File "/home/ubuntu/anaconda/lib/python2.7/site-packages/metamind/api/utils.py", line 34, in make_request
    return validate_result(response)
  File "/home/ubuntu/anac

In [94]:
preds = pd.DataFrame([
    classifier.predict(fg, input_type="files") for fg in fg_test_set
])

/home/ubuntu/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/home/ubuntu/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
/home/ubuntu/anaconda/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:79: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL 

In [102]:
pred_dicts = []

for d in preds.values:
    pred_dicts.append(d[0])

In [103]:
preds = pd.DataFrame(pred_dicts)

In [112]:
preds.groupby("label").label.count()

label
fg       153
punt      25
Name: label, dtype: int64